# Generates Mobile User Table by merging User Profile API, RT API and Daily API from YinzCam

## Description

This notebook generates a team 'user table' by merging Realtime Data, User Profile API and Daily Historical API from YinzCam.

## Preprocessing

##### Load the User Profile API data from the ADL

##### Load the Real Time API data from the ADL (`action`, `sessions`, `hardward`, `geoip`)
* join the Real Time data tables into a single `actions_sessions` table
* aggregate the Realtime data by `yinzid` and `device_id` to consolidate actions/sessions by user - find the unique `yinzid` and discard the multiple `device_id` associated with a unique `yinzid`, and we find the last login of such `yinzid`.

##### Load the Daily Historical API data from the ADL (`pvs`)

##### Merge Daily Historical with Realtime API
* outer join from Realtime & Daily `Yinzid`, first seen (min from both tables), last seen (max from both tables), device ID (deliminated list)
* For devices that do not have `yinzid` associated, add them to the list as well with same columns- first seen, last seen.

##### Merge Daily Historical/Realtime API with User Profile API
* Merge all three to one single table

## Outputs

SQL Table to MS SQL
* `team_user_table` table: Yinzid, first name, last name, first login (from janrain dictionary),first seen, last seen. Where `team in {nhl,tfc,nba}`

## QA

* Jose Nandez, Data Scientist, Jose.Nandez@MLSE.com (Primary)
* Nicole Ridout, Data Engineer, Nicole.Ridout@MLSE.com
* Farah Bastien, Manager of Data Science, Farah.Bastien@MLSE.com


##### Load the necessary functions from `PySpark` and `Python`

In [0]:
#SQL-like functions from PySpark
from pyspark.sql.functions import col,date_format,from_utc_timestamp, unix_timestamp, sum, count,countDistinct, month,min,max,when,collect_set
from pyspark.sql.functions import least, greatest, size, isnan, datediff,concat, broadcast
from pyspark.sql.types import TimestampType, IntegerType, StringType, StructType, StructField, FloatType

#python packages
from time import time, sleep
from datetime import datetime, date, timedelta
import requests, json
import pandas as pd
import numpy as np
import os, sys

##### Load the ADL configuration

In [0]:
url = "https://login.microsoftonline.com/{0}/oauth2/token".format(dbutils.secrets.get(scope = "adl_cred", key = "directory_id"))
spark.conf.set("dfs.adls.oauth2.access.token.provider.type", "ClientCredential")
spark.conf.set("dfs.adls.oauth2.client.id", dbutils.secrets.get(scope = "adl_cred", key = "client_id"))
spark.conf.set("dfs.adls.oauth2.credential", dbutils.secrets.get(scope = "adl_cred", key = "credential"))
spark.conf.set("dfs.adls.oauth2.refresh.url", url)

##### Define the team

In [0]:
dbutils.widgets.text("team", "","")
dbutils.widgets.get("team")
team = getArgument("team")
print("Working with {0} team".format(team))

Working with nhl team

## Load the User Profile API from ADL

In [0]:
def remove_zone_(x):
  try:
    return x.replace(" +0000","")
  except:
    return None
remove_zone = udf(remove_zone_,StringType())

In [0]:
adlurl = "adl://mlse1.azuredatalakestore.net/yinz_cam/"+ team +"_tor/users/"

yinzcamUsers = (spark.read.csv(adlurl,header=True)
                  .withColumn("firstLogin",remove_zone(col("firstLogin")).cast(TimestampType()))
                 ).cache()

  

In [0]:
print(yinzcamUsers.drop_duplicates(['yinzid']).count())
# display(yinzcamUsers)

283934

## Load the Realtime API

##### Define the team and url from ADL

In [0]:
adlurl = "adl://mlse1.azuredatalakestore.net/yinz_cam/"+ team +"_tor/realtime_api/"
# print(adlurl)

##### Load Realtime data from ADL

In [0]:
actions  = (spark.read.csv(adlurl + 'actions',header=True)
            .drop_duplicates()
            .withColumnRenamed('id','action_id')
            .withColumn('request_date_time',from_utc_timestamp(col('request_date_time').cast(TimestampType()), "America/Toronto"))
            .withColumn('invisible_date_time',from_utc_timestamp(col('invisible_date_time').cast(TimestampType()), "America/Toronto"))
            .withColumn('action_date',date_format('request_date_time', 'yyyy-MM-dd'))
           )
sessions = (spark.read.csv(adlurl + 'sessions',header=True)
            .withColumnRenamed('id','ses_id')
            .withColumn('start_date_time', from_utc_timestamp(col('start_date_time').cast(TimestampType()), "America/Toronto"))
            .withColumn('end_date_time', from_utc_timestamp(col('end_date_time').cast(TimestampType()), "America/Toronto"))
            .withColumn('session_date',date_format('start_date_time', 'yyyy-MM-dd'))
            .orderBy('end_date_time',ascending=False)
            .drop_duplicates(subset=['ses_id'])
            .where(col('ses_id').isNotNull())
            .withColumn("hardware_device_id",col("hardware_device_id").cast(IntegerType()))
           )
hardware = (spark.read.csv(adlurl + 'hardware',header=True)
            .withColumnRenamed('id','hardware_id')
            .drop_duplicates(subset=['hardware_id'])
            .where(col('hardware_id').isNotNull())
            .withColumn("hardware_id",col("hardware_id").cast(IntegerType()))
           )
geoip    = (spark.read.csv(adlurl + 'geoip' ,header=True)
            .withColumnRenamed('id','geoip_id')
            .withColumn('geoip_id',col('geoip_id').cast(IntegerType()))
            .where(col('geoip_id').isNotNull())
           )
geoip    =  (geoip
             .orderBy(geoip.columns, ascending=[False for i in range(len(geoip.columns))])
             .drop_duplicates(subset=['geoip_id'])
            )

##### Joining the realtime data into a single table
`PySpark` is lazy, therefore, I want to cache the table, otherwise, everytime I call `actions_sessions` the previous cell will be partially executed

In [0]:

session_join = (actions.join(sessions,col('session_id') == col('device_generated_id')))

geo_join = session_join.join(geoip,col('session_device_generated_id') == col('device_generated_id'))

# The hardware table is a very small table about 5000 records, and the other table is 300 million. So we use "Broadcast Join" to optimize the join on this situation
actions_sessions = broadcast(hardware).join(geo_join,col('hardware_id')==col('hardware_device_id'))
count = actions_sessions.cache().count()
print(count)
# print(actions_sessions.printSchema())

963349189


### Aggregating the Realtime data by `yinzid` and `device_id`

We find the unique `yinzid` and discard the multiple `device_id` associated with a unique `yinzid`, and we find the last login of such `yinzid`.

In [0]:
uniqueYinzID = (actions_sessions
                .where(col("yinzid").isNotNull())
                .orderBy(["yinzid","request_date_time"],ascending=False)
                .drop_duplicates(subset=['yinzid'])
                .select("yinzid","device_id")
                .join(actions_sessions
                      .where(col("yinzid").isNull() & col("device_id").isNotNull())
                      .select("device_id")
                      .distinct(),
                      "device_id",
                      "outer"
                     )
                .withColumn("yinzid",when(col("yinzid").isNotNull(),col("yinzid")).otherwise("EMPTY"))
                .drop_duplicates()
               )#.cache()
rtAggDF = (actions_sessions
           .withColumn("yinzid",when(col("yinzid").isNotNull(),col("yinzid")).otherwise("EMPTY"))
           .join(uniqueYinzID,["yinzid","device_id"])
           .groupBy(["yinzid","device_id"])
           .agg(min("request_date_time").alias("first_seen_l"),
                max("request_date_time").alias("last_seen_l")
               )
          )#.cache()
agg_count = rtAggDF.cache().count()
print(agg_count)

908986

In [0]:
# In general, to find second_last_seen we do this by finding all the sessions for each user and take the max request_date_time. Each user will then have
# multiple sessions and a time of their last interaction. Find the max request_date_time of each users session and remove it from table, leaving the session with
# second max request_date_time

# List of all users actions
temp = (actions_sessions
           .withColumn("yinzid",when(col("yinzid").isNotNull(),col("yinzid")).otherwise("EMPTY"))
           .join(uniqueYinzID,["yinzid","device_id"]))

# Group all users actions by session_id','yinzid','device_id' and find last action time
max_sessions = (temp.groupBy(['session_id','yinzid','device_id'])
         .agg({'request_date_time':'max'})
         .withColumnRenamed('max(request_date_time)','request_date_time')
         )
# Find the session_id that has the max request_date_time and remove it from max_sessions leaving the second session_id as the next max
first = (max_sessions.groupBy(['yinzid','device_id'])
         .agg({'request_date_time':'max'})
         .withColumnRenamed('max(request_date_time)','request_date_time'))

max_sessions_2 = max_sessions.join(first, ['yinzid','device_id','request_date_time'],'leftanti')
# Getting the second session
second = (max_sessions_2.groupBy(['yinzid','device_id'])
         .agg({'request_date_time':'max'})
         .withColumnRenamed('max(request_date_time)','second_last_seen_l'))
# Add second_last_seen_l 
rtAggDF = rtAggDF.join(second, ['yinzid','device_id'],'left')

## Load the Daily Historical API Data

In [0]:
adlurlDaily = "adl://mlse1.azuredatalakestore.net/yinz_cam/"+ team +"_tor/daily_api/pvs/"

In [0]:
pvsDF  = (spark.read.csv(adlurlDaily,header=True)
          .drop_duplicates()
          .withColumn('timestamp',from_utc_timestamp(col('timestamp').cast(TimestampType()), "America/Toronto"))
         )
pvs_count = pvsDF.cache().count()
print(pvs_count)

61042770

In [0]:
uDailyYinzid = (pvsDF
                .where(col("yinz_id").isNotNull())
                .orderBy(["yinz_id","timestamp"],ascending=False)
                .drop_duplicates(subset=['yinz_id'])
                .select("yinz_id","install_id")
                .join(pvsDF
                      .where(col("yinz_id").isNull() & col("install_id").isNotNull())
                      .select("install_id")
                      .distinct(),
                      "install_id",
                      "outer"
                     )
                .withColumn("yinz_id",when(col("yinz_id").isNotNull(),col("yinz_id")).otherwise("EMPTY"))
                .drop_duplicates()
               )#.cache()
dailyAggDF = (pvsDF
           .withColumn("yinz_id",when(col("yinz_id").isNotNull(),col("yinz_id")).otherwise("EMPTY"))
           .join(uDailyYinzid,["yinz_id","install_id"])
           .groupBy(["yinz_id","install_id"])
           .agg(min("timestamp").alias("first_seen_r"),
                max("timestamp").alias("last_seen_r")
               )
          )#.cache()

In [0]:
tmp = (pvsDF
       .withColumn("yinz_id",when(col("yinz_id").isNotNull(),col("yinz_id")).otherwise("EMPTY"))
       .join(uDailyYinzid,["yinz_id","install_id"]))

max_sessions_pvs = (tmp.groupBy(['session_id','yinz_id','install_id'])
         .agg({'timestamp':'max'})
         .withColumnRenamed('max(timestamp)','timestamp')
         )

first_pvs = (max_sessions_pvs.groupBy(['yinz_id','install_id'])
         .agg({'timestamp':'max'})
         .withColumnRenamed('max(timestamp)','timestamp'))

max_sessions_pvs_2 = max_sessions_pvs.join(first_pvs, ['yinz_id','install_id','timestamp'],'leftanti')

second_pvs = (max_sessions_pvs_2.groupBy(['yinz_id','install_id'])
         .agg({'timestamp':'max'})
         .withColumnRenamed('max(timestamp)','second_last_seen_r'))

dailyAggDF = dailyAggDF.join(second_pvs, ['yinz_id','install_id'],'left')

## Merge Daily Historical with Realtime API

In [0]:
rtDailyMerge = (rtAggDF
                .join(dailyAggDF,
                      [rtAggDF['yinzid'] == dailyAggDF["yinz_id"],
                       rtAggDF['device_id'] == dailyAggDF["install_id"]],
                      "outer"
                     )
                .withColumn("first_seen",least("first_seen_l","first_seen_r"))
                .withColumn("last_seen",greatest("last_seen_l","last_seen_r"))
                .withColumn("second_last_seen",greatest("second_last_seen_l","second_last_seen_r"))
                .withColumn("yinzid",greatest("yinzid","yinz_id"))
                .withColumn("device_id",greatest("device_id","install_id"))
                .select("yinzid","first_seen","last_seen","second_last_seen","device_id")
                .orderBy("last_seen",ascending=False)
                .drop_duplicates(subset=["yinzid","device_id"])
               )
uniqueMerged = (rtDailyMerge
                .where(col("yinzid")!="EMPTY")
                .orderBy(["yinzid","last_seen"],ascending=False)
                .drop_duplicates(subset=['yinzid'])
                .select("yinzid","device_id")
                .join(rtDailyMerge
                      .where(col("yinzid")=="EMPTY")
                      .select("device_id")
                      .distinct(),
                      "device_id",
                      "outer"
                     )
                .withColumn("yinzid",when(col("yinzid").isNotNull(),col("yinzid")).otherwise("EMPTY"))
                .drop_duplicates()
               ).cache()
rtDailyMerge = (rtDailyMerge
                .join(uniqueMerged,["yinzid","device_id"])
               )#.cache()
dailyMerge_count = rtDailyMerge.cache().count()
print(dailyMerge_count)

953925

## Merging (Realtime,Daily Historical) with User Profile API

In [0]:
 userTable = (rtDailyMerge
             .join(yinzcamUsers,'yinzid','outer')
             .withColumn("device_no_null",when(col("device_id").isNotNull(),col("device_id")).otherwise("EMPTY"))
             .withColumn("yinzid_no_null",when(col("yinzid").isNotNull(),col("yinzid")).otherwise("EMPTY"))
             .withColumn("mlse_id",concat("device_no_null","yinzid_no_null"))
             .select('mlse_id', 'first_name', 'last_name', 'email', 'first_seen', 'last_seen','second_last_seen', 'firstLogin',  
                     'clientId', 'yinzid', 'device_id', 'id_global' )
             .drop_duplicates(subset=['mlse_id'])
            )
userTable_count = userTable.cache().count()
print(userTable_count)

960974

In [0]:
# display(userTable.where((col('first_seen') > "2022-04-26") & (col('first_seen') <= "2022-04-27")))

mlse_id,first_name,last_name,email,first_seen,last_seen,second_last_seen,firstLogin,clientId,yinzid,device_id,id_global
54ADF54F-A55F-40FF-8CFA-1C932AEADA5051c8fba9-c972-49d2-b8f9-41baa20885f4,null,null,null,2022-04-26T15:47:39.000+0000,2022-04-26T22:18:27.000+0000,2022-04-26T20:20:22.000+0000,null,null,51c8fba9-c972-49d2-b8f9-41baa20885f4,54ADF54F-A55F-40FF-8CFA-1C932AEADA50,null
58AAE05C-D0CC-4301-9B3E-1C97EB0D62CF19e9ea1f-1de1-4b71-86d4-c0cea0134707,null,null,null,2022-04-26T07:24:09.000+0000,2022-04-27T10:00:35.000+0000,2022-04-26T07:25:23.000+0000,null,null,19e9ea1f-1de1-4b71-86d4-c0cea0134707,58AAE05C-D0CC-4301-9B3E-1C97EB0D62CF,null
5E2B682E-E490-4EEC-8DBA-FC96800D47BD88bdcb6b-979d-493a-8bc3-c2a4aea6ce7f,null,null,null,2022-04-26T18:41:19.000+0000,2022-04-26T18:49:42.000+0000,2022-04-26T18:49:40.000+0000,null,null,88bdcb6b-979d-493a-8bc3-c2a4aea6ce7f,5E2B682E-E490-4EEC-8DBA-FC96800D47BD,null
6FEF9F6D-8736-44C0-980B-563FC43AC0C8c0a3320e-f60f-4482-b723-39b3de3355dd,null,null,null,2022-04-26T08:57:25.000+0000,2022-04-26T18:20:32.000+0000,2022-04-26T08:57:25.000+0000,null,null,c0a3320e-f60f-4482-b723-39b3de3355dd,6FEF9F6D-8736-44C0-980B-563FC43AC0C8,null
91059B62-EA66-4728-AA79-337FC7FA468593004521-0454-4980-968a-ea31075e399b,null,null,null,2022-04-26T16:36:30.000+0000,2022-04-26T18:36:13.000+0000,2022-04-26T16:42:26.000+0000,null,null,93004521-0454-4980-968a-ea31075e399b,91059B62-EA66-4728-AA79-337FC7FA4685,null
BB9AC56A-47B4-413D-96BF-763C856AA65BEMPTY,null,null,null,2022-04-26T19:34:55.000+0000,2022-04-26T19:39:00.000+0000,2022-04-26T19:34:59.000+0000,null,null,EMPTY,BB9AC56A-47B4-413D-96BF-763C856AA65B,null
D56A58B5-A6E4-4CB3-93CD-AB3F97FF6CA9EMPTY,null,null,null,2022-04-26T20:09:18.000+0000,2022-04-26T20:09:38.000+0000,2022-04-26T20:09:18.000+0000,null,null,EMPTY,D56A58B5-A6E4-4CB3-93CD-AB3F97FF6CA9,null
D95EC91B-00D2-49F9-8F03-5A19EB6D9E4AEMPTY,null,null,null,2022-04-26T21:24:54.000+0000,2022-04-27T14:27:17.000+0000,2022-04-26T21:25:51.000+0000,null,null,EMPTY,D95EC91B-00D2-49F9-8F03-5A19EB6D9E4A,null
197181b0-5611-448a-a80f-9eb1f3a9e0a9EMPTY,null,null,null,2022-04-26T20:15:45.000+0000,2022-04-26T20:15:45.000+0000,null,null,null,EMPTY,197181b0-5611-448a-a80f-9eb1f3a9e0a9,null
2C757669-6422-4914-9016-855AD6539F5BEMPTY,null,null,null,2022-04-26T21:05:12.000+0000,2022-04-26T21:39:24.000+0000,2022-04-26T21:14:05.000+0000,null,null,EMPTY,2C757669-6422-4914-9016-855AD6539F5B,null



## Send Merged User Table to SQL

In [0]:
sqlserver = dbutils.secrets.get(scope = "jdbc", key = "sqlserver")
port = '1433'
database = 'mlse_sqldb'
user = dbutils.secrets.get(scope = "jdbc", key = "username")
pswd = dbutils.secrets.get(scope = "jdbc", key = "password")
url = 'jdbc:sqlserver://' + sqlserver + ':' + port + ';database=' + database

In [0]:
# Overwrites the previous table
(userTable
 .coalesce(8)
 .write
 .option('user', user)
 .option('password', pswd)
 .jdbc(url, team.lower() + '_user_table', mode = 'overwrite' )
)